In [1]:
# imports
import pandas as pd
import numpy as np
import random
import statistics as stat

# traindf = pd.read_pickle("./pickles/df_small_clean.pkl")

# display(traindf[traindf["position"] == 11])

In [2]:
def random_solution(df):
    
    # csv_name = "./results/" + csv_name
    
    # get list of hotels per search id
    results_per_search = df.groupby('srch_id').size()
    
    column_values = df["srch_id"].values
    
    ids =  pd.unique(column_values) 
    
    all_ranks = []
    
    for i, ideetje in enumerate(ids):
        df_temp = df[df["srch_id"] == ideetje]
        
        ranks = df_temp["position"].values
        
        # shuffle ranks
        random.shuffle(ranks)
        all_ranks.extend(ranks)

    df["predicted_rank"] = all_ranks
    return df

In [3]:
def indiv_search_ndcg_score(true_rank, predicted_rank, booked, clicked):
    # holder for ideal score
    idcg = 0
    
    # holder for our score
    dcg = 0

    # itereer over elk resultaat in de search
    for i in range(len(true_rank)):
        # hotels die geboekt zijn zijn heel belangrijk (5)
        if booked[i] == 1:
            
            # increment ideal score
            if i != 0:
                idcg = idcg + (5 / np.log2(i+1))
            else:
                idcg += 5

            # if predicted well, increment score
            if true_rank[i] == predicted_rank[i]:
                if i != 0:
                    dcg = dcg + (5 / np.log2(i+1))
                else:
                    dcg += 5
                
        # hotels waarop geklikt is zijn een beetje belangrijk (1)
        elif clicked[i] == 1:

            # increment ideal score
            if i != 0:
                idcg = idcg + (1 / np.log2(i+1))
            else:
                idcg += 1
            
            # if predicted well, increment score
            if true_rank[i] == predicted_rank[i]:
                dcg = dcg + (1 / np.log2(i+2))

    if idcg == 0:
        return dcg
    return dcg/idcg

In [4]:
# score function for every search (thus iterate over groupby object)
def dataframe_ndcg_score(solution):
    all_scores = []
    for srchid, group in solution.groupby('srch_id'):

        # true_rank
        true_rank = np.asarray(group['position'])

        # predicted_rank
        predicted_rank = np.asarray(group['predicted_rank'])
        
        # booking_bool
        booked = np.asarray(group['booking_bool'])
        
        # click_bool
        clicked = np.asarray(group['click_bool'])
        all_scores.append(indiv_search_ndcg_score(true_rank, predicted_rank, booked, clicked))
        
    return np.nanmean(all_scores)

In [5]:
# for i in range(5):
#     print(i, dataframe_ndcg_score(random_solution(traindf)))

In [23]:
rf = pd.read_pickle("./pickles/XGBoost_solution.pkl")

print(round(dataframe_ndcg_score(rf),3) * 100, "%")

13.4 %


In [18]:
rf.head(5)

,prop_id,srch_id,position,price_quality,competitor_lower,competitor_available,visited_before,click_bool,booking_bool,price_usd,promotion_flag,prop_brand_bool,random_bool,prop_location_score1,prop_location_score2,prop_id_price_mean,importance,predicted_rank
2680787,7453,179893,20,36.333333,1,0,0,0,0,109.00,0,0,1,1.39,0.156319,47.240556,0,35
1682390,65391,112976,2,22.333333,1,1,0,0,0,67.00,1,0,0,2.56,0.001800,24.704231,0,21
3587156,90380,240946,12,69.750000,0,0,0,0,0,279.00,0,0,1,4.17,0.078900,48.852857,0,3
2808379,80693,188584,22,39.500000,0,0,0,0,0,79.00,0,1,0,0.00,0.002200,37.583333,0,30
3610826,101579,242565,20,27.207500,0,0,0,0,0,108.83,0,1,1,4.48,0.121800,31.752500,0,33
